In [ ]:
import findspark
findspark.init()
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.mongodb.spark:mongo-spark-connector_2.11:2.2.0 --conf "spark.mongodb.input.uri=mongodb://127.0.0.1/test.myCollection?readPreference=primaryPreferred" --conf "spark.mongodb.output.uri=mongodb://127.0.0.1/test.myCollection" pyspark-shell'
from pyspark import SparkConf, SparkContext
import collections
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions
import sys, time 
sys.path.append(os.path.abspath("E:\mongo-hadoop-master\mongo-hadoop-master\spark\src\main\python"))
import pymongo_spark
pymongo_spark.activate()
import numpy as np
from pyspark.sql.functions import lit

In [ ]:
results=[]

In [ ]:
ms = SparkSession.builder.appName('Test').config("spark.mongodb.input.uri", "mongodb://127.0.0.1/TestDB.TestCollection").config("spark.mongodb.output.uri", "mongodb://127.0.0.1/TestDB.TCollection").getOrCreate()

In [ ]:
RC = [20,40,60,80,100]
OC = [2000,4000,6000,8000,10000]

In [ ]:
#Update Heavy (A)
for r in RC:
    df = ms.read.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1/TestDB.M%s"%r).load()
    cc=[]
    qq=np.array(df.toPandas().values)
    for n in np.unique(qq[:,4]):
        m = str(n)
        cc.append(m[m.find("'")+1:m.find("'",m.find("'")+1)])
    for o in OC:
        df.write.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1/TestDB.Temp").mode('overwrite').save()
        np.random.seed(1)
        st = time.time()
        for i in range(int(o/2)):
            pipeline = "{'$match': {'_id': ObjectId('%s')}}"%cc[np.random.randint(r*10000)]
            vv=ms.read.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1/TestDB.M%s"%r).option("pipeline", pipeline).load()
            gj = vv.rdd
            gj.cache()
            vv=vv.withColumn('_3',lit('0'))
            vv.write.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1/TestDB.Temp").mode('append').save()
        et = time.time()-st
        results.append(['A',r,o,et])
print("A")

In [ ]:
#Read Heavy (B)
for r in RC:
    df = ms.read.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1/TestDB.M%s"%r).load()
    cc=[]
    qq=np.array(df.toPandas().values)
    for n in np.unique(qq[:,4]):
        m = str(n)
        cc.append(m[m.find("'")+1:m.find("'",m.find("'")+1)])
    for o in OC:
        wp=o/20
        df.write.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1/TestDB.Temp").mode('overwrite').save()
        np.random.seed(1)
        st = time.time()
        l = 0
        for i in range(int(o*0.95)):
            pipeline = "{'$match': {'_id': ObjectId('%s')}}"%cc[np.random.randint(r*10000)]
            vv=ms.read.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1/TestDB.M%s"%r).option("pipeline", pipeline).load()
            gh = vv.rdd
            gh.cache()
            if l%wp ==0:
                vv=vv.withColumn('_3',lit('0'))
                vv.write.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1/TestDB.Temp").mode('append').save()
            l+=1
        et = time.time()-st
        results.append(['B',r,o,et])
print("B")

In [ ]:
#Read Only (C)
for r in RC:
    df = ms.read.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1/TestDB.M%s"%r).load()
    cc=[]
    qq=np.array(df.toPandas().values)
    for n in np.unique(qq[:,4]):
        m = str(n)
        cc.append(m[m.find("'")+1:m.find("'",m.find("'")+1)])
    for o in OC:
        np.random.seed(1)
        st = time.time()
        for i in range(o):
            pipeline = "{'$match': {'_id': ObjectId('%s')}}"%cc[np.random.randint(r*10000)]
            vv=ms.read.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1/TestDB.M%s"%r).option("pipeline", pipeline).load()
            gh = vv.rdd
            gh.cache()
        et = time.time()-st
        results.append(['C',r,o,et])
print("C")

In [ ]:
#Read Latest (D)
for r in RC:
    df = ms.read.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1/TestDB.M%s"%r).load()
    cc=[]
    qq=np.array(df.toPandas().values)
    for n in np.unique(qq[:,4]):
        m = str(n)
        cc.append(m[m.find("'")+1:m.find("'",m.find("'")+1)])
    for o in OC:
        wp=o/20
        np.random.seed(1)
        st = time.time()
        l = 0
        for i in range(int(o*0.95)):
            pipeline = "{'$match': {'_id': ObjectId('%s')}}"%cc[l]
            vv=ms.read.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1/TestDB.M%s"%r).option("pipeline", pipeline).load()
            gh = vv.rdd
            gh.cache()
            if l%wp ==0:
                vv.write.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1/TestDB.Temp2").mode('overwrite').save()
            l+=1
        et = time.time()-st
        results.append(['D',r,o,et])
print("D")

In [ ]:
#Short Ranges (E)
for r in RC:
    df = ms.read.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1/TestDB.M%s"%r).load()
    cc=[]
    qq=np.array(df.toPandas().values)
    for n in np.unique(qq[:,4]):
        m = str(n)
        cc.append(m[m.find("'")+1:m.find("'",m.find("'")+1)])
    for o in OC:
        print(o,r)
        wp=o/20
        np.random.seed(1)
        k=0
        st = time.time()
        for i in range(int(o*0.95)):
            l = np.random.randint(r*10000-10)
            for u in range(l,l+10):
                pipeline = "{'$match': {'_id': ObjectId('%s')}}"%cc[u]
                vv=ms.read.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1/TestDB.M%s"%r).option("pipeline", pipeline).load()
                gh = vv.rdd
                gh.cache()
            gh.cache()
            if k%wp ==0:
                vv.write.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1/TestDB.Temp2").mode('overwrite').save()
            l+=1
            k+=1
        et = time.time()-st
        results.append(['E',r,o,et])
print("E")

In [ ]:
np.savetxt('DResults1m.csv',np.array(results),delimiter=',',fmt='%s')